In [1]:
import pandas as pd, numpy as np
from influxdb import DataFrameClient
import json
# import feedparser

Set params

In [2]:
purge=False
# purge=True

In [3]:
process_annoations=False
# process_annoations=True

In [4]:
# process_macro=False
process_macro=True

In [5]:
# write_out=False
write_out=True

In [6]:
full_overwrite=False
# full_overwrite=True

Load pre-params

In [7]:
df0=pd.read_csv('df0.csv').set_index('date')
df0.index=pd.to_datetime(df0.index)

In [8]:
bets=json.loads(open('bets.json','r').read())

In [9]:
severity=pd.read_csv('severity.csv').set_index('date')
severity.index=pd.to_datetime(severity.index)

In [10]:
mobility=pd.read_csv('mobility.csv').set_index('date')
mobility.index=pd.to_datetime(mobility.index)

In [11]:
mobility_mini=pd.read_csv('mobility_mini.csv').set_index('date')
mobility_mini.index=pd.to_datetime(mobility_mini.index)

Establish DB connection

In [12]:
user = 'admin'
password = open('auth/influxa.txt','r').read()
host='influxdb'
port=8086
dbname='base'
dbname_long='long'
protocol = 'line' #'json'
client = DataFrameClient(host, port, user, password, dbname)
client_long = DataFrameClient(host, port, user, password, dbname_long)

In [13]:
if purge:
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '600d', 1, default=True)
    client_long.drop_database(dbname_long)
    client_long.drop_retention_policy(dbname_long)
    client_long.create_database(dbname_long)
    client_long.create_retention_policy(dbname_long, '6000d', 1, default=True)

In [14]:
htmlipath='../html/'
htmlepath='//covid-large-html.csaladen.es/'
htmlepath_other='//covid-html.csaladen.es/'

In [15]:
titles={'HU':"Magyar",'RO':'Română','EN':'English'}

In [16]:
rtitles={titles[t]:t for t in titles}

In [17]:
import requests
grafana = "http://grafana:3000/"
headers = {
    'Authorization': 'Bearer '+open('auth/grafana.txt','r').read(),
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [18]:
response = requests.get(grafana+'api/folders', headers=headers)
folders=json.loads(response.content)

In [19]:
folder_id=[f['id'] for f in folders if f['title']=='COVID-19 Romania'][0]
# folder_id=[f['id'] for f in folders if f['title']=='COVID-19 Romania Old'][0]
# folder_id=[f['id'] for f in folders if f['title']=='Testing area'][0]

In [20]:
response = requests.get(grafana+'api/search?folderIds='+str(folder_id), headers=headers)
dashs=json.loads(response.content)

In [21]:
uids={rtitles[d['title']]:d['uid'] for d in dashs if d['title'] in rtitles}
iids={rtitles[d['title']]:d['id'] for d in dashs if d['title'] in rtitles}

In [22]:
uids_light={rtitles[d['title'].split(' Light')[0]]:d['uid'] for d in dashs if 'Light' in d['title']}
iids_light={rtitles[d['title'].split(' Light')[0]]:d['id'] for d in dashs if 'Light' in d['title']}

In [23]:
# response = requests.get(grafana+'api/dashboards/uid/hu-test', headers=headers)
# model=json.loads(response.content)['dashboard']
# open('hu-test.json','w').write(json.dumps(model))

In [24]:
languages=['HU','RO','EN']

In [25]:
url='https://docs.google.com/spreadsheets/d/'+open('auth/sheet.txt','r').read()+'/gviz/tq?tqx=out:csv&sheet='

In [26]:
def get_szotar():
    sheet='szotar'
    columns=languages+[i+'_description' for i in languages]+[i+'_source' for i in languages]
    df=pd.read_csv(url+sheet)
    df=df[['ID','UI']+columns]
    sheet='minidashboard'
    df2=pd.read_csv(url+sheet)
    df2=df2[['ID','UI']+columns]
    df=pd.concat([df,df2])
    szotardf=df.set_index('ID')[columns]
    szotar=df.set_index('ID').T.to_dict()
    szotarHU=df.set_index('HU',drop=False).T.to_dict()
    szotarRO=df.set_index('RO',drop=False).T.to_dict()
    szotarEN=df.set_index('EN',drop=False).T.to_dict()
    return szotardf,szotar,szotarHU,szotarRO,szotarEN

In [27]:
def resolve_time_conflicts(series):
    ds={}
    ts=[]
    for d in series:
        if d not in ds:
            ds[d]=pd.to_datetime(d)
            t=(ds[d])
        else:
            ds[d]=ds[d]+pd.to_timedelta('193m')
            t=(ds[d])
        ts.append(t)
    return ts

# Influx

In [28]:
szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

<ipython-input-26-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-26-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-26-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [29]:
import time
import datetime
import pytz
utc=pytz.UTC

In [34]:
def push2influx(df,measurement,field_columns,tag_columns,shift=False,dbclient=client,wo=write_out,fo=full_overwrite,daily=True):
    if wo:
        df=df.sort_index()
        # df.index = utc.localize(df.index) 
        df.index=df.index.tz_localize('GMT')#.tz_convert('Europe/Bucharest')
        if shift:
            df.index+=pd.to_timedelta('12h')
        if fo: 
            print('Purging',measurement,'...')
            dbclient.query('DROP MEASUREMENT '+measurement)
        else:
            latest=dbclient.query('SELECT * FROM '+measurement+' GROUP BY "1d" ORDER BY DESC LIMIT 1')
            if latest:
                lat=latest[list(latest.keys())[0]].index[0]
                if daily: lat+=pd.to_timedelta('1d')
                df=df[lat:]
                print('Slicing',measurement,'from',lat,'...')
            else:
                print('No data in db for',measurement,'...')
        time.sleep(3)
        print('Writing to',measurement,'...')
        bsize=5000
        bwait=2
        print(len(df),'data points will be written in',len(df)/bsize,'batches.')
        print('Expected query running time is:',int((len(df)/bsize)*bwait*1.1)+3,'seconds.')
        for i in range(int(len(df)/bsize)+1):
            r=range(i*bsize,min(len(df),(i+1)*bsize))
            dc=df.iloc[r]
            print('Writing batch',i+1,'...')
            dbclient.write_points(dc, measurement, protocol=protocol,
                                field_columns=field_columns,
                                tag_columns=[])
            time.sleep(bwait)
        time.sleep(3)
        print('Done!')
    else:
        print('Write-out not enabled. Skipping...')

DateLaZi

In [35]:
#new method from DateLaZi
dlz=requests.get('https://datelazi.ro/latestData.json').content
# dlz=requests.get('https://di5ds1eotmbx1.cloudfront.net/latestData.json').content
# dlz=requests.get('https://d35p9e4fm9h3wo.cloudfront.net/latestData.json').content
dlz=json.loads(dlz)
dlz['historicalData'][dlz['currentDayStats']['parsedOnString']]=dlz['currentDayStats']
dlzs=[]
dlz_counties=[]
for date in dlz['historicalData']:
    d={}
    mdate=date.replace('2018-11-07','2020-11-07').replace('2020-01-05','2021-01-05').replace('2020-01-07','2021-01-07')
    d['date']=mdate
    d['cases']=dlz['historicalData'][date]['numberInfected']
    d['heals']=dlz['historicalData'][date]['numberCured']
    d['deaths']=dlz['historicalData'][date]['numberDeceased']
    if 'vaccines' in dlz['historicalData'][date]:
        if dlz['historicalData'][date]['vaccines']:
            for v in dlz['historicalData'][date]['vaccines']:
                for k in ['total_administered','immunized']:
                    k2=k.replace('immunized','total_immunized')
                    if k2 not in d: d[k2]=0
                    d[k2]+=dlz['historicalData'][date]['vaccines'][v][k]
                    d[k2+'_'+v]=dlz['historicalData'][date]['vaccines'][v][k]
    if 'countyInfectionsNumbers' in dlz['historicalData'][date]:
        counties=dlz['historicalData'][date]['countyInfectionsNumbers']
        if counties:
            for county in counties:
                dummy={'date':mdate,county:dlz['historicalData'][date]['countyInfectionsNumbers'][county]}
                if county=='PH':
                    if date=='2020-12-01':
                        dummy[county]=19395
                dlz_counties.append(dummy)
    dlzs.append(d)
dl=pd.DataFrame(dlzs).set_index('date').sort_index(ascending=True)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#Manual fixes
dl.loc['2021-04-08','cases']=993613
dl.loc['2021-04-08','heals']=890528
dl.loc['2021-04-08','deaths']=24733

In [ ]:
for c in dl.columns[3:]:
    dl[c]=dl[c].cumsum()
dl2=dl.diff()
dl2.columns=['case','heal','death']+[i.replace('total_','') for i in dl2.columns[3:]]
dl['active']=dl['cases']-dl['heals']-dl['deaths']
dl=dl.join(dl2)

In [ ]:
latest=dl.index[-1]
now=str(pd.to_datetime("now"))[:10]
print(latest,now)

Tests from `graphs.ro`

In [ ]:
r=requests.get('https://www.graphs.ro/json.php')
graphs_ro=json.loads(r.content)['covid_romania']
g=pd.DataFrame(graphs_ro)
# g['date']=pd.to_datetime(g['reporting_date'])
g['date']=g['reporting_date']
g=g.set_index('date')
g['tests']=g['total_tests']
g['test']=g['new_tests_today']
latest2=g.index[0]
print(latest2)

In [ ]:
latest=str(min(pd.to_datetime(latest),pd.to_datetime(latest2)))[:10]
print(latest)

In [ ]:
dl=dl.join(g[['tests','test']])

Governance

In [32]:
sheet='governance'

In [33]:
df=pd.read_csv(url+sheet)
# dl=pd.concat([df.set_index('date').sort_index()[:'2020-03-17'][dl.columns[:-1]],
#           dl['2020-03-18':]])
df=df.set_index('date').sort_index(ascending='True').join(dl,lsuffix='_g')#[20:] #[:-1] vaccines nincs a governance-ban

NameError: name 'dl' is not defined

In [ ]:
# fill from local, if missing in DateLaZi
for c in ['active','cases','heals','deaths','case','heal','death']:
    df[c]=df[c].fillna(df[c+'_g'].astype(str).str.replace(',','')).replace('nan',np.nan)
df=df.dropna(how='all',axis=0).dropna(how='all',axis=1)

In [ ]:
df14=df[['cases']].reset_index().join(df[['cases']][14:].reset_index(),lsuffix='1')
df14['case14']=df14['cases'].astype(float)-df14['cases1'].astype(float)
df=df.join(df14.set_index('date')['case14'].dropna().astype(int))

In [ ]:
df.index=pd.to_datetime(df.index)
df0=df.copy()
#df['date']=pd.to_datetime(df['date'])
#df=df[df.columns[:13]].set_index('date')
vaccine_totals=[i for i in list(dl2.columns[3:])+list(dl.columns[3:len(dl2.columns)]) if 'total_' in i]
df=pd.DataFrame(df[['active','cases','heals','deaths','case14']+\
                  vaccine_totals].stack()).reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float).astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']
current=df[df['type']=='case14']['value'][-1] #sajat becsles
# current=df[df['type']=='active']['value'][-1] #official
current

In [ ]:
df0.to_csv('df0.csv')

In [172]:
#sanity check
cols=[ 'active','case14', 'case', 'heal', 'death','cases', 'heals', 'deaths']
dx=df0[cols].dropna(how='all')
dx=dx.loc[~dx.index.isnull()]
dx[cols].tail()

,active,case14,case,heal,death,cases,heals,deaths
date,,,,,,,,
2021-04-08,78352.0,73819.0,4989.0,4818.0,172.0,993613.0,890528.0,24733.0
2021-04-09,77115.0,72245.0,4942.0,6045.0,134.0,998555.0,896573.0,24867.0
2021-04-10,75620.0,70686.0,4310.0,5666.0,139.0,1002865.0,902239.0,25006.0
2021-04-11,74373.0,69549.0,3302.0,4420.0,129.0,1006167.0,906659.0,25135.0
2021-04-12,72944.0,68047.0,2323.0,3639.0,113.0,1008490.0,910298.0,25248.0


-> Run till here to test Grafana only.

# Grafana

In [173]:
today0=now[-2:]
if today0[0]=='0': 
    today=today0[1]
else:
    today=today0
currents=str(current)[:-3]+'&nbsp;'+str(current)[-3:]
hour=str(pd.to_datetime('now')+pd.to_timedelta('3h'))[11:16]
str(szotar['report']['UI'][1:]),current,today0,hour

('412', 68047, '12', '01:03')

In [174]:
def darken2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0xfefefe >> 1))[2:]
def brighten2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0x7f7f7f << 1))[2:]
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.rgb2hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [175]:
# szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

In [176]:
colors_to_darken=['#F2CC0C','#CA95E5','#FF780A','#E0B400','#96D98D','#F2495C','#E02F44',
                  'lime','#73BF69','#8AB8FF','#3274D9','#D3D3D3','#FF7383','#FADE2A']

# Maps

In [177]:
top=3

In [178]:
# iso_counties=pd.read_html('https://en.wikipedia.org/wiki/Counties_of_Romania')[1]\
#     .set_index(['ISO code[note 3]'])['County'].to_dict()
# iso_counties['BN']='Bistrița-Năsăud'
# iso_counties['CL']='Călărași'
# open(htmlipath+'panels/iso_counties.json','w').write(json.dumps(iso_counties))
iso_counties=json.loads(open(htmlipath+'panels/iso_counties.json','r').read())

In [179]:
georo=json.loads(open(htmlipath+'panels/romania-counties.json','r').read())
georoco={i['properties']['NAME_1']:i['properties']['ID_1'] for i in georo['objects']['ROU_adm1']['geometries']}   
georoco['București']=georoco['Bucharest']

Locality map

In [180]:
# datagov1a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/bc19c354-644d-4a24-a26f-512129dbc70d/download/transparenta_vaccinare_martie_2021.xlsx')
datagov2a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d0b60b45-fb08-4980-a34c-8cbb4a43cad3/download/transparenta_martie_2021.xlsx',skiprows=1)
# datagov1b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/a0eb9ff2-9b97-430c-b285-360cadb55307/download/vaccinare-covid19-grupe-risc-01-01.04.2021.xlsx')
datagov2b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d3280256-07cc-4f93-957a-9815085899be/download/transparenta_aprilie_2021.xlsx',skiprows=1)

In [181]:
# datagov1=pd.concat([datagov1a,datagov1b])
datagov2=datagov2a.set_index(['Judet','UAT']).join(datagov2b.set_index(['Judet','UAT'])).reset_index()

In [182]:
# del datagov1a
# del datagov1b
del datagov2a
del datagov2b

In [183]:
uat=json.loads(open(htmlipath+'panels/uat_simplificat.geojson','r').read())

In [184]:
locs=[]
for f in uat['features']:
    locs.append(f['properties'])
locs=pd.DataFrame(locs)[['judet','uat','siruta']]

In [185]:
def judetconverter(c):
    return (' '.join([i.capitalize() for i in c.split(' ')])).replace('ţ','ț').replace('ş','ș').replace('năsăud','Năsăud').\
        replace('severin','Severin').replace('mare','Mare').\
        replace('Municipiul București','Bucharest').replace('București','Bucharest')

In [186]:
geo_iso=pd.DataFrame(iso_counties,index=['index']).T.reset_index().set_index('index').join(\
    pd.DataFrame(georoco,index=['geo_id']).T.reset_index().set_index('index')).reset_index()
geo_iso.columns=['en','iso2','geo_id']

In [187]:
locs['judet_norm']=[judetconverter(i) for i in locs['judet']]
datagov2['judet_norm']=[judetconverter(i) for i in datagov2['Judet']]

In [188]:
def uatconverter(judet,c):
    c=(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in c.split(' ')]))\
        .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
        .replace('î','â').replace(' De ',' de ')\
        .replace('Cuza Voda','Cuza Vodă')
    
    if (judet=='Dolj'):
        c=c.replace('Segarcea','Șegarcea')
    elif (judet=='Teleorman'):
        c=c.replace('Turnu Magurele','Turnu Măgurele')
    elif (judet=='Olt'):
        c=c.replace('Ipotesti','Ipotești')
    elif (judet=='Alba'):
        c=c.replace('Râmetea','Rimetea')
    elif (judet=='Constanța'):
        c=c.replace('44066','')
    elif (judet=='Ialomița'):
        c=c.replace('Radulești','Rădulești')
    elif (judet=='Argeș'):
        c=c.replace('Ciofringeni','Ciofrângeni')\
        .replace('Valea Mare Pravăț','Valea Mare-Pravăț')
    elif (judet=='Vâlcea'):
        c=c.replace('Păusești-Măglași','Păușești-Măglași')
    elif (judet=='Prahova'):
        c=c.replace('Cocorastii Colt','Cocorăștii Colț')
    elif (judet=='Bucharest'):
        c=c.replace('București Sectorul','Sector')
    elif (judet=='Gorj'):
        c=c.replace('Crușet','Crușeț')
    elif (judet=='Brăila'):
        c=c.replace('Racoviță','Racovița')\
        .replace('Gradiștea','Grădiștea')\
        .replace('Chișcani','Chiscani')
    elif (judet=='Cluj'):
        c=c.replace('Râșca','Rișca')
    elif (judet=='Botoșani'):
        c=c.replace('Răușeni','Răuseni')
    elif (judet=='Maramureș'):
        c=c.replace('Șisești','Șișești')
    elif (judet=='Brașov'):
        c=c.replace('Sambata de Sus','Sâmbăta de Sus')
    elif (judet=='Vaslui'):
        c=c.replace('Tătărăni','Tătărani')
    elif (judet=='Iași'):
        c=c.replace('Țigănăși','Țigănași')
    elif (judet=='Bistrița-Năsăud'):
        c=c.replace('Ilva Mica','Ilva Mică')
    elif (judet=='Bihor'):
        c=c.replace('Sănnicolau Romăn','Sânnicolau Român')
    elif (judet=='Mureș'):
        c=c.replace('Sarmașu','Sărmașu')
    return c

In [189]:
datagov2['uat_norm']=[(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in u.split(' ')]))\
                      .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
                      .replace(' De ',' de ').replace('Municipiul ','').replace('Oraș ','')\
                      for u in datagov2['UAT']]

In [190]:
uat_norms=[]
for judet in locs['judet_norm'].unique():
    for l in locs[locs['judet_norm']==judet]['uat'].unique():
        u=uatconverter(judet,str(l))
        if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
            u=u.replace(' ','-')
            if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
                print(judet,l,u)
                u=''
        uat_norms.append({'judet_norm':judet,'uat_norm':u,'uat':l})

Constanța 44066 
  


In [191]:
locs2=locs.set_index(['judet_norm','uat']).join(pd.DataFrame(uat_norms).set_index(['judet_norm','uat'])).reset_index()

In [192]:
datagov3=datagov2.set_index(['judet_norm','uat_norm']).join(locs2.set_index(['judet_norm','uat_norm']))
datagov3=datagov3.set_index(['judet','uat','siruta'],append=True).drop(['UAT','Judet'],axis=1)
datagov3.index=datagov3.index.reorder_levels([2,3,4,0,1])
# datagov3_dates=[pd.to_datetime(c) for c in datagov3.columns]
# datagov3_dates=[c for c in datagov3.columns]
datagov3_dates=[str(pd.to_datetime(c)) for c in datagov3.columns]

In [193]:
datagov3=datagov3.round(2)

In [194]:
# del datagov1
del datagov2

In [195]:
def cat(r):
    if r<1: return '< 1 ‰'
    elif r<2: return '1-2 ‰'
    elif r<4: return '2-4 ‰'
    elif r<6: return '4-6 ‰'
    elif r<8: return '6-8 ‰'
    elif r<10: return '8-10 ‰'
    elif r<20: return '> 10-20 ‰'
    else: return '> 20 ‰'
def cat2(r):
    if r<1: return 0
    elif r<2: return 1
    elif r<4: return 2
    elif r<6: return 3
    elif r<8: return 4
    elif r<10: return 5
    elif r<20: return 6
    else: return 7

In [196]:
t=0
new_uat={'type':'FeatureCollection','features':[]}
new_uat_numbers=[]
new_uat_daily={'type':'FeatureCollection','features':[]}
for i in range(len(uat['features'])):
    dummy={}
    judet=uat['features'][i]['properties']['judet']
    if (judet!=''):
        u=uat['features'][i]['properties']['uat']
        siruta=uat['features'][i]['properties']['siruta']
        # dummy={'judet':judet,'uat':u,'siruta':siruta}
        if u in datagov3.loc[judet].index:
            d=datagov3.loc[judet].loc[u].loc[siruta]
            values=d.values[0]
            dummy['🌄']=d.index[0][0]
            dummy['🏠']=d.index[0][1]
            for t in range(len(values)):
                value=values[t]
                date=datagov3_dates[t]
                dummy['📈']=value
                dummy['🔴']=cat(value)
                dummy['⚫']=cat2(value)
                dummy['📆']=date
                feature={'type':'Feature','geometry':uat['features'][i]['geometry'],'properties':dummy.copy()}
                new_uat['features'].append(feature)
                new_uat_numbers.append(dummy.copy())
                if (t==len(datagov3_dates)-1):
                    new_uat_daily['features'].append(feature)

In [197]:
open(htmlipath+'panels/new_uat_numbers.json','w').write(json.dumps(new_uat_numbers))
# open(htmlipath+'panels/new_uat.json','w').write(json.dumps(new_uat))

22431835

In [198]:
#Kepler.gl export
open(htmlipath+'panels/daily/new_uat'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/new_uat_daily.json','w').write(json.dumps(new_uat_daily))

7100483

In [199]:
del new_uat

In [200]:
#D3plus export

In [201]:
datagov4=datagov3.stack().reset_index()
datagov4.columns=['judet','uat','id','judet_norm','uat_norm','date','value']
datagov4['value']=np.round(datagov4['value'],2)
# datagov4=datagov4.dropna(subset=['judet'])
datagov4=datagov4.dropna(how='any')

In [202]:
open(htmlipath+'panels/daily/uat2b_'+date[:10]+'.json','w').write(json.dumps(list(datagov4.T.to_dict().values())))

20881697

In [203]:
# #HTML overwrite
# cases_localities=open(htmlipath+'panels/cases_localities.html','r').read()
# cases_localities=cases_localities[:cases_localities.find('daily/uat2b_\' + \'2021-')+17]+latest+cases_localities[cases_localities.find('daily/uat2b_\' + \'2021-')+27:]
# open(htmlipath+'panels/cases_localities.html','w').write(cases_localities)
# #### CHECK IF LATEST DATE IS ONE LESS than latest

Restrictions

In [204]:
df2=pd.DataFrame(new_uat_numbers)
df4=df2[['🌄', '🏠', '📈','📆']].set_index(['🌄', '🏠', '📆']).unstack()['📈'].sort_index(axis=1)

In [205]:
def restrictie(x,w):
    if x>7.5: return 4
    elif x>7:
        if w>7.5: return 3
        else: return 2
    elif x>4: return 2
    elif x>3.5:
        if w>4: return 1
        else: return 0
    else:
        return 0
    
def restrictie2(x,w):
    if x>7.5: return '7.5+'
    elif x>7:
        if w>7.5: return '7.5-'
        else: return '4+'
    elif x>4: return '4+'
    elif x>3.5:
        if w>4: return '4-'
        else: return '0'
    else:
        return '0'

In [206]:
dr=[]
for i in df4.T.iteritems():
    v=i[1]
    #print(i[0])
    for j in range(len(v)):
        t=v.index[j]
        x=v[j]
        y=v[j-min(14,j):j].values
        w=x
        if len(y):
            w=max(y)
        r=restrictie(x,w)
        #print(j,t,x,w,r)
        dr.append({'🌄':i[0][0], '🏠':i[0][1], '🚦':r,'📆':t})

In [207]:
dp=pd.DataFrame(dr).set_index(['🌄', '🏠','📆']).astype(float)

In [208]:
for i in new_uat_daily['features']:
    i['properties']['🚦']=dp.loc[i['properties']['🌄']].loc[i['properties']['🏠']].loc[i['properties']['📆']]['🚦']

In [237]:
open(htmlipath+'panels/new_uat_daily2.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/daily/new_uat_daily2_'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))
open('uat_daily.json','w').write(json.dumps(new_uat_daily))

7167347

In [210]:
#HTML overwrite
uat_html=open(htmlipath+'panels/daily/mapbox-'+date[:10]+'.html','r').read()
uat_html=uat_html.replace(date[5:10],date[5:8]+today0).replace(date[5:10],date[5:8]+today0)
uat_html=uat_html[:uat_html.find('.json')-2]+\
    str(pd.to_datetime(now)+pd.to_timedelta('-1d'))[8:10]+\
    uat_html[uat_html.find('.json'):]
open(htmlipath+'panels/daily/mapbox-'+date[:8]+today0+'.html','w').write(uat_html)

18658

Upload to `uat.json` to mapbox

In [84]:
#Mapbox colors:
# 239e69
# f5d232
# c52b69
# 850237

In [85]:
#Add szekelydata GitHub update
#Add mapbox direct upload

In [211]:
#Sanity check
datagov3.loc['Cluj'].loc['Cluj-Napoca'].T.tail(3)

siruta,54975
judet_norm,Cluj
uat_norm,Cluj-Napoca
2021-04-09,7.51
2021-04-10,7.11
2021-04-11,7.07


Mapbox upload

In [226]:
import requests

Create tileset source

In [281]:
!tilesets validate-source uat_daily.json

Validating features
✔ valid


In [283]:
!tilesets upload-source csaladenes roeim uat_daily.json --replace --token $mapbox_api_key


{"id": "mapbox://tileset-source/csaladenes/roeim", "files": 1, "source_size": 6792276, "file_size": 6792276}


In [285]:
%%capture cap_out
!tilesets publish csaladenes.roeim --token $mapbox_api_key

In [293]:
jobid=json.loads(cap_out.stdout.split('}')[0]+'}')['jobId']
!tilesets job csaladenes.roeim $jobid --token $mapbox_api_key

{"id": "cknfbky1w002e08mj7pa9hqm0", "stage": "success", "created": 1618275721316, "created_nice": "Tue Apr 13 2021 01:02:01 GMT+0000 (Coordinated Universal Time)", "published": 1618275721316, "tilesetId": "csaladenes.roeim", "errors": [], "warnings": [], "completed": 1618275831852, "layer_stats": {"uat_fills": {"minzoom": 0, "polygon_count": 63460, "total_tiles": 7328, "capped": 0, "zooms": {"0": {"capped": 0, "ymin": 0, "avg_size": 275, "tiles": 1, "xmin": 0, "xmax": 0, "max_size": 275, "sum_area": 508164394.24620897, "sum_size": 275, "ymax": 0, "size_histogram": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, "1": {"capped": 0, "ymin": 0, "avg_size": 4681, "tiles": 1, "xmin": 1, "xmax": 1, "max_size": 4681, "sum_area": 127041098.56155224, "sum_size": 4681, "ymax": 0, "size_histogram": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, "2": {"capped": 0, "ymin": 1, "avg_size": 80184, "tiles": 1, "xmin": 2, "xmax": 2, "max_size": 80184, "sum_area": 58476014.2

Wait 5 min

Push to Grafana

In [72]:
nevuto={'0':'é','1':'é','2':'é','3':'á','4':'é','5':'é','6':'á','7':'é','8':'á','9':'é'}
for lang in languages:
# for lang in ['EN']:
    response = requests.get(grafana+'api/dashboards/uid/'+lang.lower(), headers=headers)
    model=json.loads(response.content)['dashboard']
    for i in model['panels']:
        if lang=='HU':
            #print(i['id'],i['type'],i['title'])
            if i['id'] in (179, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find('n,')-4].strip()+' '+today+'-'+\
                    nevuto[now[-1]]+i['description'][i['description'].find('n,'):\
                    i['description'].find(':')-2]+hour+\
                    i['description'][i['description'].find(':')+3:]
                i['title']=i['title'][:i['title'].rfind(' ')]+' '+today
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='EN':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' on '+today+' '+ i['description'][i['description'].find('on ')+5:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+4:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='RO':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' în data de '+today+' '+ i['description'][i['description'].find('în data de ')+14:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+4:]
                
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('<b>')+3]+today0+\
                i['content'][i['content'].rfind('<b>')+5:]
            
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })       
    print(r,r.content)
    open(lang+'.json','w').write(json.dumps(model))    
    
    model_light=json.dumps(model).replace('dark','light',9999).replace('lightGray','#52545C',9999)\
        .replace('#d3d3d3','#52545c',9999)\
        .replace('csaladen.es/favicon.ico" style="','csaladen.es/favicon.ico" style="filter: brightness(0.3);',9999)
    for color in colors_to_darken:
        model_light=model_light.replace(color,adjust_lightness(color,0.8),9999)
        model_light=model_light.replace(color.lower(),adjust_lightness(color,0.8),9999)
    model=json.loads(model_light)
    model['title']=titles[lang]+' Light'
    model['uid']=uids_light[lang]
    model['id']=iids_light[lang]
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })    
    print(r,r.content)
    open(lang+'-light.json','w').write(json.dumps(model))   

<Response [200]> b'{"id":2,"slug":"magyar","status":"success","uid":"hu","url":"/d/hu/magyar","version":311}'
<Response [200]> b'{"id":5,"slug":"magyar-light","status":"success","uid":"hu-light","url":"/d/hu-light/magyar-light","version":136}'
<Response [200]> b'{"id":3,"slug":"romana","status":"success","uid":"ro","url":"/d/ro/romana","version":215}'
<Response [200]> b'{"id":6,"slug":"romana-light","status":"success","uid":"ro-light","url":"/d/ro-light/romana-light","version":130}'
<Response [200]> b'{"id":4,"slug":"english","status":"success","uid":"en","url":"/d/en/english","version":176}'
<Response [200]> b'{"id":7,"slug":"english-light","status":"success","uid":"en-light","url":"/d/en-light/english-light","version":131}'
